# LSTM + ResNet Stress Classifier

End-to-end exploration of a convolutional + recurrent model for stress-level prediction.

In [1]:

import os
import math
from pathlib import Path
from typing import List, Tuple

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold
from sklearn.metrics import classification_report, accuracy_score

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from imblearn.over_sampling import SMOTE

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [2]:

# Configuration
DEFAULT_DATASET_ROOT = Path("./Datasets")
DATASET_ROOT = Path(os.getenv("DATASET_ROOT", DEFAULT_DATASET_ROOT))
STATES = ["STRESS", "AEROBIC", "ANAEROBIC"]
TARGET_FS = 4.0
WINDOW_SECONDS = 60
WINDOW_STEP_SECONDS = 30
MIN_LABEL_COVERAGE = 0.6
SEED = 42
MAX_SUBJECTS = None  # limit per state if desired
APPLY_CHANNEL_NORMALIZATION = True
APPLY_DIFF_CHANNELS = True
APPLY_SMOTE = True

np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)


In [3]:

# Helper functions for Empatica-format data
STRESS_STAGE_ORDER_S = ["Stroop", "TMCT", "Real Opinion", "Opposite Opinion", "Subtract"]
STRESS_STAGE_ORDER_F = ["TMCT", "Real Opinion", "Opposite Opinion", "Subtract"]
STRESS_TAG_PAIRS_S = [(3, 4), (5, 6), (7, 8), (9, 10), (11, 12)]
STRESS_TAG_PAIRS_F = [(2, 3), (4, 5), (6, 7), (8, 9)]
STRESS_PHASES = {"Stroop", "TMCT", "Real Opinion", "Opposite Opinion", "Subtract"}
STRESS_LEVEL_BOUNDS = {"low": 3.0, "moderate": 6.0}
STRESS_LEVEL_PHASE_BOUNDS = {
    "Stroop": {"low": 2.5, "moderate": 5.0},
    "Opposite Opinion": {"low": 2.5, "moderate": 5.5},
    "Real Opinion": {"low": 2.8, "moderate": 5.5},
    "TMCT": {"low": 2.8, "moderate": 5.8},
    "Subtract": {"low": 2.8, "moderate": 5.8},
}
STRESS_LEVEL_FILES = ["Stress_Level_v1.csv", "Stress_Level_v2.csv"]
MIN_LABEL_COVERAGE = 0.6


def load_stress_levels():
    levels = {}
    for fname in STRESS_LEVEL_FILES:
        path = Path(fname)
        if not path.exists():
            continue
        df = pd.read_csv(path, index_col=0)
        df.columns = [str(c).strip() for c in df.columns]
        for subject, row in df.iterrows():
            subj = str(subject).strip()
            levels[subj] = {
                col: (float(row[col]) if not pd.isna(row[col]) else np.nan)
                for col in df.columns
            }
    return levels


STRESS_LEVELS = load_stress_levels()


def base_subject_id(subject: str) -> str:
    return subject.split("_")[0]


def read_signal(path: Path):
    with open(path, "r") as f:
        start_line = f.readline().strip()
        if not start_line:
            raise ValueError(f"Missing start timestamp in {path}")
        start_ts = pd.to_datetime(start_line.split(",")[0])
        fs_line = f.readline().strip()
        if not fs_line:
            raise ValueError(f"Missing sample rate in {path}")
        fs = float(fs_line.split(",")[0])
        data = np.genfromtxt(f, delimiter=",")
    data = np.asarray(data, dtype=float)
    if data.ndim == 0:
        data = data.reshape(1, 1)
    return fs, data.squeeze(), start_ts


def read_tags(path: Path, start_ts: pd.Timestamp):
    if not path.exists():
        return []
    df = pd.read_csv(path, header=None)
    tags = []
    for ts_str in df[0].astype(str):
        ts = pd.to_datetime(ts_str)
        tags.append((ts - start_ts).total_seconds())
    return [(t, t) for t in tags]


def stress_intervals_from_tags(tags, subject):
    if not tags:
        return []
    times = [t for t, _ in tags]
    if subject.startswith("S"):
        idx_pairs = STRESS_TAG_PAIRS_S
        stage_order = STRESS_STAGE_ORDER_S
    else:
        idx_pairs = STRESS_TAG_PAIRS_F
        stage_order = STRESS_STAGE_ORDER_F
    base_id = base_subject_id(subject)
    spans = []
    for stage, (i, j) in zip(stage_order, idx_pairs):
        if i < len(times) and j < len(times) and times[j] > times[i]:
            level = STRESS_LEVELS.get(base_id, {}).get(stage)
            spans.append({"start": times[i], "end": times[j], "stage": stage, "stress_level": level})
    return spans


def active_intervals_from_tags(tags):
    if len(tags) < 2:
        return []
    spans = []
    for (a, _), (b, _) in zip(tags[:-1], tags[1:]):
        if b > a:
            spans.append({"start": a, "end": b, "stage": "active", "stress_level": 0.0})
    return spans


def stress_bucket(level: float = None, phase: str = None) -> str:
    if phase in {"aerobic", "anaerobic", "rest", "active"}:
        return "no_stress"
    if level is None or pd.isna(level) or level <= 0:
        return "no_stress"
    bounds = STRESS_LEVEL_PHASE_BOUNDS.get(phase, STRESS_LEVEL_BOUNDS)
    if level <= bounds["low"]:
        return "low_stress"
    if level <= bounds["moderate"]:
        return "moderate_stress"
    return "high_stress"


def resample_to_rate(signal: np.ndarray, src_fs: float, tgt_fs: float) -> np.ndarray:
    if signal.ndim == 1:
        signal = signal[:, None]
    src_len = signal.shape[0]
    duration = src_len / src_fs
    tgt_len = int(duration * tgt_fs)
    if tgt_len <= 0:
        return np.zeros((0, signal.shape[1]))
    src_t = np.linspace(0, duration, src_len, endpoint=False)
    tgt_t = np.linspace(0, duration, tgt_len, endpoint=False)
    resampled = np.vstack([
        np.interp(tgt_t, src_t, signal[:, i])
        for i in range(signal.shape[1])
    ]).T
    if resampled.shape[1] == 1:
        return resampled[:, 0]
    return resampled


def window_intervals(duration: float, win_s: int, step_s: int):
    windows = []
    t = 0.0
    while t + win_s <= duration:
        windows.append((t, t + win_s))
        t += step_s
    return windows


def assign_label(win, intervals):
    start, end = win
    length = end - start
    best_label = None
    best_cov = 0.0
    best_span = None
    for label, spans in intervals.items():
        overlap = 0.0
        span_choice = None
        span_overlap = 0.0
        for span in spans:
            a = span["start"] if isinstance(span, dict) else span[0]
            b = span["end"] if isinstance(span, dict) else span[1]
            inter = max(0.0, min(end, b) - max(start, a))
            if inter > 0:
                overlap += inter
                if inter > span_overlap:
                    span_overlap = inter
                    span_choice = span
        coverage = overlap / length
        if coverage > best_cov:
            best_cov = coverage
            best_label = label
            best_span = span_choice
    if best_cov >= MIN_LABEL_COVERAGE and best_label is not None:
        return best_label, best_span
    return None, None


def make_label_intervals(state: str, subject: str, tags, duration: float):
    rest_span = [{"start": 0.0, "end": duration, "stage": "rest", "stress_level": 0.0}]
    if state == "STRESS":
        stress_spans = stress_intervals_from_tags(tags, subject)
        if not stress_spans:
            return {"rest": rest_span}
        return {"stress": stress_spans, "rest": rest_span}
    active = active_intervals_from_tags(tags)
    label = "aerobic" if state == "AEROBIC" else "anaerobic"
    if not active:
        return {label: rest_span, "rest": rest_span}
    return {label: active, "rest": rest_span}


def load_subject_state(state: str, subject: str):
    folder = DATASET_ROOT / state / subject
    if not folder.exists():
        raise FileNotFoundError(folder)
    fs_eda, eda_raw, start_ts = read_signal(folder / "EDA.csv")
    temp_path = folder / "TEMP.csv"
    if temp_path.exists():
        fs_temp, temp_raw, _ = read_signal(temp_path)
    else:
        fs_temp, temp_raw = fs_eda, np.zeros_like(eda_raw)
    fs_acc, acc_raw, _ = read_signal(folder / "ACC.csv")
    acc_raw = np.atleast_2d(acc_raw)
    acc_mag = np.linalg.norm(acc_raw, axis=1)
    bvp_path = folder / "BVP.csv"
    if bvp_path.exists():
        fs_bvp, bvp_raw, _ = read_signal(bvp_path)
    else:
        fs_bvp, bvp_raw = None, None
    tags = read_tags(folder / "tags.csv", start_ts)
    sensors = {
        "EDA": np.asarray(eda_raw, dtype=float),
        "TEMP": np.asarray(temp_raw, dtype=float),
        "ACC_MAG": acc_mag,
    }
    if bvp_raw is not None:
        sensors["BVP"] = np.asarray(bvp_raw, dtype=float)
    fs_map = {"EDA": fs_eda, "TEMP": fs_temp, "ACC_MAG": fs_acc}
    if fs_bvp:
        fs_map["BVP"] = fs_bvp
    duration = len(sensors["EDA"]) / fs_eda
    return {"sensors": sensors, "fs": fs_map, "tags": tags, "duration": duration}


In [4]:

EXPECTED_LEN = int(WINDOW_SECONDS * TARGET_FS)
BASE_CHANNELS = ["EDA", "TEMP", "ACC", "BVP"]


def _slice_or_pad(signal: np.ndarray, start: int, end: int) -> np.ndarray:
    length = end - start
    if signal is None or len(signal) == 0:
        return np.zeros(length, dtype=np.float32)
    if end > len(signal):
        pad = end - len(signal)
        segment = signal[start: len(signal)]
        if pad > 0:
            segment = np.concatenate([segment, np.zeros(pad, dtype=segment.dtype)])
        return segment.astype(np.float32)
    return signal[start:end].astype(np.float32)


def build_sequence_dataset(states: List[str] = STATES, max_subjects: int = 0) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    sequences = []
    labels = []
    subjects = []
    for state in states:
        state_dir = DATASET_ROOT / state
        if not state_dir.exists():
            continue
        subject_ids = sorted([p.name for p in state_dir.iterdir() if p.is_dir()])
        if max_subjects and max_subjects > 0:
            subject_ids = subject_ids[:max_subjects]
        for subj in tqdm(subject_ids, desc=f"{state}"):
            try:
                info = load_subject_state(state, subj)
            except Exception as exc:
                print(f"Skip {state}/{subj}: {exc}")
                continue
            sensors = info["sensors"]
            fs_map = info["fs"]
            tags = info["tags"]
            duration = info["duration"]

            eda = resample_to_rate(sensors["EDA"], fs_map["EDA"], TARGET_FS)
            temp = resample_to_rate(sensors.get("TEMP", np.zeros_like(eda)), fs_map.get("TEMP", TARGET_FS), TARGET_FS) if "TEMP" in sensors else np.zeros_like(eda)
            acc = resample_to_rate(sensors["ACC_MAG"], fs_map["ACC_MAG"], TARGET_FS)
            if "BVP" in sensors and "BVP" in fs_map:
                bvp = resample_to_rate(sensors["BVP"], fs_map["BVP"], TARGET_FS)
            else:
                bvp = np.zeros_like(eda)

            intervals = make_label_intervals(state, subj, tags, duration)
            windows = window_intervals(duration, WINDOW_SECONDS, WINDOW_STEP_SECONDS)

            for win in windows:
                label_name, span_meta = assign_label(win, intervals)
                if label_name is None or span_meta is None:
                    continue
                start_idx = int(round(win[0] * TARGET_FS))
                end_idx = start_idx + EXPECTED_LEN
                eda_win = _slice_or_pad(eda, start_idx, end_idx)
                temp_win = _slice_or_pad(temp, start_idx, end_idx)
                acc_win = _slice_or_pad(acc, start_idx, end_idx)
                bvp_win = _slice_or_pad(bvp, start_idx, end_idx)

                stress_stage = span_meta.get("stage") if isinstance(span_meta, dict) else None
                stress_level = span_meta.get("stress_level") if isinstance(span_meta, dict) else None
                if label_name == "stress":
                    if stress_level is None or np.isnan(stress_level):
                        continue
                else:
                    stress_level = 0.0
                phase_label = stress_stage if stress_stage else label_name
                stress_class = stress_bucket(stress_level, phase_label)

                tensor = np.stack([eda_win, temp_win, acc_win, bvp_win], axis=0)
                sequences.append(tensor)
                labels.append(stress_class)
                subjects.append(base_subject_id(subj))
    sequences = np.stack(sequences).astype(np.float32)
    labels = np.array(labels)
    subjects = np.array(subjects)
    return sequences, labels, subjects


In [5]:

sequences, labels, subjects = build_sequence_dataset(max_subjects=MAX_SUBJECTS if MAX_SUBJECTS else 0)
channel_names = BASE_CHANNELS.copy()
print("Raw sequences:", sequences.shape)
print("Label distribution before processing:", pd.Series(labels).value_counts())

if APPLY_CHANNEL_NORMALIZATION and sequences.size:
    means = sequences.mean(axis=(0, 2), keepdims=True)
    stds = sequences.std(axis=(0, 2), keepdims=True) + 1e-6
    sequences = (sequences - means) / stds

if APPLY_DIFF_CHANNELS:
    diffs = np.diff(sequences, axis=2, prepend=sequences[:, :, :1])
    sequences = np.concatenate([sequences, diffs], axis=1)
    channel_names = channel_names + [name + "_diff" for name in channel_names]

print("Processed sequences:", sequences.shape)
print("Channels:", channel_names)
print("Subjects count:", len(np.unique(subjects)))

STRESS:   0%|          | 0/37 [00:00<?, ?it/s]

Skip STRESS/f14_a: No columns to parse from file


AEROBIC:   0%|          | 0/31 [00:00<?, ?it/s]

ANAEROBIC:   0%|          | 0/32 [00:00<?, ?it/s]

Raw sequences: (6788, 4, 240)
Label distribution before processing: no_stress          5571
moderate_stress     664
high_stress         462
low_stress           91
Name: count, dtype: int64
Processed sequences: (6788, 8, 240)
Channels: ['EDA', 'TEMP', 'ACC', 'BVP', 'EDA_diff', 'TEMP_diff', 'ACC_diff', 'BVP_diff']
Subjects count: 36


In [6]:

class SequenceDataset(Dataset):
    def __init__(self, data: np.ndarray, labels: np.ndarray):
        self.features = torch.from_numpy(data)
        self.labels = torch.from_numpy(labels).long()

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]


In [7]:

class ResNetBlock(nn.Module):
    def __init__(self, channels: int, kernel_size: int = 5, dilation: int = 1):
        super().__init__()
        padding = ((kernel_size - 1) // 2) * dilation
        self.conv1 = nn.Conv1d(channels, channels, kernel_size, padding=padding, dilation=dilation)
        self.bn1 = nn.BatchNorm1d(channels)
        self.conv2 = nn.Conv1d(channels, channels, kernel_size, padding=padding, dilation=dilation)
        self.bn2 = nn.BatchNorm1d(channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += identity
        return self.relu(out)


class LSTMResNet(nn.Module):
    def __init__(self, input_channels: int, cnn_channels: int = 32, lstm_hidden: int = 64, num_classes: int = 4):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv1d(input_channels, cnn_channels, kernel_size=7, padding=3),
            nn.BatchNorm1d(cnn_channels),
            nn.ReLU(inplace=True),
        )
        self.resnet = nn.Sequential(
            ResNetBlock(cnn_channels, kernel_size=5),
            ResNetBlock(cnn_channels, kernel_size=5, dilation=2),
            nn.Dropout(0.1),
        )
        self.lstm = nn.LSTM(
            input_size=cnn_channels,
            hidden_size=lstm_hidden,
            num_layers=2,
            dropout=0.2,
            batch_first=True,
            bidirectional=True,
        )
        self.classifier = nn.Sequential(
            nn.Linear(lstm_hidden * 2, lstm_hidden),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(lstm_hidden, num_classes),
        )

    def forward(self, x):
        out = self.initial(x)
        out = self.resnet(out)
        out = out.transpose(1, 2)
        lstm_out, _ = self.lstm(out)
        last = lstm_out[:, -1, :]
        return self.classifier(last)


In [8]:

# Training configuration
EPOCHS = 20
BATCH_SIZE = 64
LR = 5e-4
N_SPLITS = 3

le = LabelEncoder()
encoded_labels = le.fit_transform(labels)
num_classes = len(le.classes_)
print("Classes:", dict(zip(le.classes_, range(num_classes))))


Classes: {np.str_('high_stress'): 0, np.str_('low_stress'): 1, np.str_('moderate_stress'): 2, np.str_('no_stress'): 3}


In [9]:

results = []
num_channels = sequences.shape[1]
gkf = GroupKFold(n_splits=N_SPLITS)
for fold, (train_idx, test_idx) in enumerate(gkf.split(sequences, encoded_labels, subjects)):
    print(f"===== Fold {fold} =====")
    X_train, X_test = sequences[train_idx], sequences[test_idx]
    y_train, y_test = encoded_labels[train_idx], encoded_labels[test_idx]

    if APPLY_SMOTE:
        flat_train = X_train.reshape(X_train.shape[0], -1)
        smote = SMOTE(random_state=SEED)
        flat_resampled, y_train = smote.fit_resample(flat_train, y_train)
        X_train = flat_resampled.reshape(-1, num_channels, EXPECTED_LEN).astype(np.float32)

    train_dataset = SequenceDataset(X_train, y_train)
    test_dataset = SequenceDataset(X_test, y_test)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    class_counts = np.bincount(y_train, minlength=num_classes)
    class_weights = (len(y_train) / (num_classes * class_counts.clip(min=1)))
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32, device=device)

    model = LSTMResNet(input_channels=num_channels, num_classes=num_classes).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
    criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=3)

    for epoch in range(1, EPOCHS + 1):
        model.train()
        train_loss = 0.0
        for xb, yb in train_loader:
            xb = xb.to(device)
            yb = yb.to(device)
            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)
            optimizer.step()
            train_loss += loss.item() * xb.size(0)
        train_loss /= len(train_loader.dataset)

        model.eval()
        val_loss = 0.0
        preds_all = []
        targets_all = []
        with torch.no_grad():
            for xb, yb in test_loader:
                xb = xb.to(device)
                yb = yb.to(device)
                logits = model(xb)
                loss = criterion(logits, yb)
                val_loss += loss.item() * xb.size(0)
                preds_all.append(torch.argmax(logits, dim=1).cpu().numpy())
                targets_all.append(yb.cpu().numpy())
        val_loss /= len(test_loader.dataset)
        preds_all = np.concatenate(preds_all)
        targets_all = np.concatenate(targets_all)
        val_acc = accuracy_score(targets_all, preds_all)
        scheduler.step(val_loss)
        print(f"Epoch {epoch:02d} | Train Loss {train_loss:.4f} | Val Loss {val_loss:.4f} | Val Acc {val_acc:.3f}")

    report = classification_report(targets_all, preds_all, target_names=le.classes_, zero_division=0, output_dict=True)
    print(pd.DataFrame(report).transpose())
    results.append({
        "fold": fold,
        "accuracy": report["accuracy"],
        "macro_f1": report["macro avg"]["f1-score"],
        "weighted_f1": report["weighted avg"]["f1-score"],
    })

results_df = pd.DataFrame(results)
print("Cross-validated metrics:")
print(results_df)
print("Mean metrics:")
print(results_df[["accuracy", "macro_f1", "weighted_f1"]].mean())


===== Fold 0 =====


/home/moh/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


Epoch 01 | Train Loss 0.7944 | Val Loss 1.0208 | Val Acc 0.619
Epoch 02 | Train Loss 0.5987 | Val Loss 1.2826 | Val Acc 0.513
Epoch 03 | Train Loss 0.5291 | Val Loss 0.8526 | Val Acc 0.712
Epoch 04 | Train Loss 0.4855 | Val Loss 0.9475 | Val Acc 0.672
Epoch 05 | Train Loss 0.4558 | Val Loss 1.1391 | Val Acc 0.606
Epoch 06 | Train Loss 0.4228 | Val Loss 0.8372 | Val Acc 0.722
Epoch 07 | Train Loss 0.3967 | Val Loss 1.0148 | Val Acc 0.674
Epoch 08 | Train Loss 0.3638 | Val Loss 0.9293 | Val Acc 0.713
Epoch 09 | Train Loss 0.3586 | Val Loss 0.9625 | Val Acc 0.743
Epoch 10 | Train Loss 0.3209 | Val Loss 1.0072 | Val Acc 0.705
Epoch 11 | Train Loss 0.2675 | Val Loss 0.9268 | Val Acc 0.730
Epoch 12 | Train Loss 0.2382 | Val Loss 0.9929 | Val Acc 0.702
Epoch 13 | Train Loss 0.2341 | Val Loss 1.1042 | Val Acc 0.713
Epoch 14 | Train Loss 0.2095 | Val Loss 1.2552 | Val Acc 0.675
Epoch 15 | Train Loss 0.1854 | Val Loss 1.1897 | Val Acc 0.703
Epoch 16 | Train Loss 0.1807 | Val Loss 1.1469 | Val Ac

/home/moh/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


Epoch 01 | Train Loss 0.9967 | Val Loss 0.8033 | Val Acc 0.672
Epoch 02 | Train Loss 0.7135 | Val Loss 0.8052 | Val Acc 0.672
Epoch 03 | Train Loss 0.6164 | Val Loss 0.8271 | Val Acc 0.674
Epoch 04 | Train Loss 0.5568 | Val Loss 0.8731 | Val Acc 0.673
Epoch 05 | Train Loss 0.5127 | Val Loss 0.7984 | Val Acc 0.715
Epoch 06 | Train Loss 0.4687 | Val Loss 0.7768 | Val Acc 0.738
Epoch 07 | Train Loss 0.4343 | Val Loss 0.8266 | Val Acc 0.722
Epoch 08 | Train Loss 0.4075 | Val Loss 0.8202 | Val Acc 0.739
Epoch 09 | Train Loss 0.3810 | Val Loss 0.8380 | Val Acc 0.740
Epoch 10 | Train Loss 0.3484 | Val Loss 0.8619 | Val Acc 0.747
Epoch 11 | Train Loss 0.2925 | Val Loss 0.9131 | Val Acc 0.742
Epoch 12 | Train Loss 0.2704 | Val Loss 0.9055 | Val Acc 0.749
Epoch 13 | Train Loss 0.2574 | Val Loss 0.9156 | Val Acc 0.761
Epoch 14 | Train Loss 0.2429 | Val Loss 0.9810 | Val Acc 0.739
Epoch 15 | Train Loss 0.2163 | Val Loss 0.9367 | Val Acc 0.757
Epoch 16 | Train Loss 0.1956 | Val Loss 0.9769 | Val Ac

/home/moh/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


Epoch 01 | Train Loss 0.9336 | Val Loss 0.7471 | Val Acc 0.679
Epoch 02 | Train Loss 0.7010 | Val Loss 1.0230 | Val Acc 0.591
Epoch 03 | Train Loss 0.6027 | Val Loss 0.9919 | Val Acc 0.602
Epoch 04 | Train Loss 0.5562 | Val Loss 0.8156 | Val Acc 0.722
Epoch 05 | Train Loss 0.5199 | Val Loss 0.9270 | Val Acc 0.662
Epoch 06 | Train Loss 0.4222 | Val Loss 0.7781 | Val Acc 0.764
Epoch 07 | Train Loss 0.3993 | Val Loss 0.8779 | Val Acc 0.723
Epoch 08 | Train Loss 0.3958 | Val Loss 0.8631 | Val Acc 0.732
Epoch 09 | Train Loss 0.3582 | Val Loss 0.8686 | Val Acc 0.728
Epoch 10 | Train Loss 0.3281 | Val Loss 0.8357 | Val Acc 0.748
Epoch 11 | Train Loss 0.3143 | Val Loss 0.8896 | Val Acc 0.749
Epoch 12 | Train Loss 0.3111 | Val Loss 0.9019 | Val Acc 0.738
Epoch 13 | Train Loss 0.2888 | Val Loss 0.9388 | Val Acc 0.737
Epoch 14 | Train Loss 0.2656 | Val Loss 0.8963 | Val Acc 0.765
Epoch 15 | Train Loss 0.2656 | Val Loss 0.9513 | Val Acc 0.753
Epoch 16 | Train Loss 0.2611 | Val Loss 0.9756 | Val Ac